In [1]:
import cv2
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import kagglehub
import shutil

In [2]:
def augmentation_image(image_path):
    image_bgr = cv2.imread(image_path)
    augmented = image_bgr.copy()
    h, w = image_bgr.shape[:2]

    # Danh sách các biến đổi khả thi
    transformations = ['rotate', 'flip', 'shift', 'scale', 'brightness', 'noise']
    
    # Chọn ngẫu nhiên ít nhất 3 biến đổi
    num_transforms = random.randint(3, len(transformations))
    chosen = random.sample(transformations, num_transforms)

    for t in chosen:
        if t == 'flip':
            flip_code = random.choice([-1, 0, 1])  # ngang, dọc, cả hai
            augmented = cv2.flip(augmented, flip_code)

        elif t == 'shift':
            tx = random.randint(-20, 20)
            ty = random.randint(-20, 20)
            M = np.float32([[1, 0, tx], [0, 1, ty]])
            augmented = cv2.warpAffine(augmented, M, (w, h))

        elif t == 'brightness':
            alpha = random.uniform(0.8, 1.2)  # contrast
            beta = random.randint(-30, 30)    # brightness
            augmented = cv2.convertScaleAbs(augmented, alpha=alpha, beta=beta)

        elif t == 'noise':
            noise = np.random.normal(0, 1, augmented.shape).astype(np.int16)
            augmented = augmented.astype(np.int16) + noise
            augmented = np.clip(augmented, 0, 255).astype(np.uint8)

    return augmented

In [3]:
base_dir = r'/kaggle/input/dataset-augment/Lymphocite'
image_path = base_dir + '/kaggle/input/dataset-augment/Lymphocite/1.JPG'

# Import Dataset & Test

Image size: (640, 640, 3)
Data type: uint8


(-0.5, 639.5, 639.5, -0.5)

In [4]:
# Lấy danh sách ảnh trong thư mục test
image_files = [f for f in os.listdir(base_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp'))]

# Chọn 1 ảnh đầu tiên để test
if not image_files:
    raise FileNotFoundError("Không tìm thấy ảnh nào trong thư mục test/")
    
image_path = os.path.join(base_dir, image_files[1])

# Đọc ảnh gốc
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

print(f"Image size: {image_bgr.shape}")
print(f"Data type: {image_bgr.dtype}")

plt.imshow(image_rgb)
plt.title(f"Ảnh gốc")
plt.axis("off")

In [5]:
#augmentation
image_path = os.path.join(base_dir, image_files[0])
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
aug_img = augmentation_image(image_path)

# Hiển thị ảnh gốc
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image_rgb)
plt.title(f"Ảnh gốc")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(aug_img)
plt.title('Image augmentated')
plt.axis('off')

plt.tight_layout()
plt.show()

# Create augmented image into folder

In [6]:
output_dir = "augmentation_image"
os.makedirs(output_dir, exist_ok=True)

In [7]:
ground_truth_dict = {
    1: 89,
    2: 60,
    3: 60,
    4: 44,
    5: 54,
    6: 68,
    7: 48,
    8: 44,
    9: 52,
    10: 46,
    11: 41,
    12: 58,
    13: 36,
    14: 36,
    15: 46,
    16: 60,
    17: 236,
    18: 258,
    19: 365,
    20: 874,
    21: 913,
    22: 740,
    23: 838,
    24: 422,
    25: 401,
    26: 386,
    27: 319,
    28: 261,
    29: 300,
    30: 300,
    31: 300,
    32: 99,
    33: 43,
    34: 298,
    35: 300,
}

In [8]:
for img_name in image_files:
    img_path = os.path.join(base_dir, img_name)

    stt = int(os.path.splitext(img_name)[0])

    # tra ground truth theo dict
    gt_value = ground_truth_dict[stt]

    for i in range(3):
        aug_img = augmentation_image(img_path)

        base_name = os.path.splitext(img_name)[0]
        save_name = f"{base_name}_GT{gt_value}_aug{i+1}.jpg"
        save_path = os.path.join(output_dir, save_name)
        cv2.imwrite(save_path, aug_img)

print("Augmentation complete, image saved in folder:", output_dir)

Augmentation complete, image saved in folder: augmentation_image


In [9]:
#Up load dataset
LOCAL_DATASET_DIR = "/kaggle/working/augmentation_image"

shutil.make_archive("augmentation_image", "zip", LOCAL_DATASET_DIR)

kagglehub.login()

DATASET_SLUG = "lymphocyte-augmentation-final"

kagglehub.dataset_upload(
    handle=f"giahuytranviet/{DATASET_SLUG}",
    local_dataset_dir=LOCAL_DATASET_DIR,
    version_notes="Lymphocite Dataset augmented ."
)

print("Dataset is upload completely on Kaggle Hub.")

Uploading Dataset https://www.kaggle.com/datasets/giahuytranviet/lymphocyte-augmentation-final ...
More than 50 files detected, creating a zip archive...
Starting upload for file /tmp/tmpkgesd2sr/archive.zip


Uploading: 100%|██████████| 8.04M/8.04M [00:00<00:00, 10.8MB/s]

Upload successful: /tmp/tmpkgesd2sr/archive.zip (8MB)


Your dataset has been created.
Files are being processed...
See at: https://www.kaggle.com/datasets/giahuytranviet/lymphocyte-augmentation-final
Dataset is upload completely on Kaggle Hub.
